In [ ]:
import sys
sys.path.extend(['../../'])

import os
import time
from src.utils.io import *
from src.utils.recon import *

In [ ]:
old_root = '../../data/raw/ChallengeData/SingleCoil/Cine/TrainingSet/FullSample'
new_root = '../../data/transformed/SingleCoil/Cine/TrainingSet'
old_postfix = '.mat'
new_postfix = '.h5'
old_label = 'kspace_single_full'
new_label = 'kspace'
data_files = ['cine_lax', 'cine_sax']

if not os.path.exists(new_root):
    os.makedirs(new_root)

for subject in os.listdir(old_root):
    if os.path.isdir(os.path.join(old_root, subject)):
        print(f'Start processing subject {subject} ...')
        start_time = time.perf_counter()
        for data_file in data_files:
            old_filename = os.path.join(old_root, subject, data_file + old_postfix)
            new_filename = os.path.join(new_root, subject + '_' + data_file + new_postfix)
            if os.path.exists(old_filename):
                kspace = h5read(old_filename, old_label)
                kspace = kspace['real'] + 1j * kspace['imag']
                h5write(new_filename, new_label, kspace)
        print(f'Finish processing subject {subject}: {time.perf_counter() - start_time: .3f} seconds.')

In [ ]:
old_root = '../../data/raw/ChallengeData/MultiCoil/Cine/TrainingSet/FullSample'
new_root = '../../data/transformed/MultiCoil/Cine/TrainingSet'
old_postfix = '.mat'
new_postfix = '.h5'
old_label = 'kspace_full'
new_label = 'kspace'
data_files = ['cine_lax', 'cine_sax']

if not os.path.exists(new_root):
    os.makedirs(new_root)

for subject in os.listdir(old_root):
    if os.path.isdir(os.path.join(old_root, subject)):
        print('Start processing subject ' + subject + '...')
        start_time = time.perf_counter()
        for data_file in data_files:
            old_filename = os.path.join(old_root, subject, data_file + old_postfix)
            new_filename = os.path.join(new_root, subject + '_' + data_file + new_postfix)
            if os.path.exists(old_filename):
                kspace = h5read(old_filename, old_label)
                kspace = kspace['real'] + 1j * kspace['imag']
                avg_kspace = np.average(kspace, axis=0)
                num_slices = avg_kspace.shape[0]
                sens = np.zeros_like(avg_kspace)
                for i in range(num_slices):
                    sens[i] = espirit_map(avg_kspace[i])
                h5write(new_filename, new_label, kspace)
                h5write(new_filename, 'sens', sens)
        print('Finish processing subject ' + subject + ': ' + str(time.perf_counter() - start_time))